In [61]:
from pulp import *

In [66]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus

def tsp_with_extra_constraints(n, cost_matrix, constraints):
    assert len(cost_matrix) == n, f'Cost matrix is not {n}x{n}'
    assert all(len(cj) == n for cj in cost_matrix), f'Cost matrix is not {n}x{n}'
    assert all( 1 <= i < n and 1 <= j < n and i != j for (i,j) in constraints)
    
    prob = LpProblem("TSP_with_constraints", LpMinimize)
    
    # Create binary variables for edges
    x = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                x[(i, j)] = LpVariable(f"edge_{i}_{j}", cat='Binary')
    
            
    # Create variables for vertex positions in tour (u[i] represents position of vertex i)
    u = {}
    for i in range(1, n):
        u[i] = LpVariable(f"pos_{i}", lowBound=1, upBound=n-1, cat='Integer')
    
    # Objective function: minimize total cost
    prob += lpSum(cost_matrix[i][j] * x[i,j]
                 for i in range(n)
                 for j in range(n)
                 if i != j)
    
    # Constraint: each vertex must be entered exactly once
    for j in range(n):
        prob += lpSum(x[i,j] for i in range(n) if i != j) == 1
    
    # Constraint: each vertex must be exited exactly once
    for i in range(n):
        prob += lpSum(x[i,j] for j in range(n) if i != j) == 1
    
    # Subtour elimination constraints (MTZ formulation)
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                prob += u[i] - u[j] + n * x[i,j] <= n - 1
    
    # Add precedence constraints
    for (i, j) in constraints:
        if i != 0 and j != 0:  # Skip if either vertex is the start/end point
            prob += u[i] <= u[j] - 1
    
    # Solve the problem
    prob.solve()
    
    # Extract the tour
    tour = [0]  # Start at vertex 0
    current = 0
    for _ in range(n-1):
        for j in range(n):
            if j != current and value(x[current,j]) > 0.5:
                tour.append(j)
                current = j
                break
                
    return tour

In [67]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
constraints = [(3,4),(1,2)]
tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
i = 0
tour_cost = 0
for j in tour[1:]:
    tour_cost += cost_matrix[i][j]
    i = j
tour_cost += cost_matrix[i][0]
print(f'Tour:{tour}')
print(f'Cost of your tour: {tour_cost}')
assert abs(tour_cost-10) <= 0.001, 'Expected cost was 10'
for i in range(n):
    num = sum([1 if j == i else 0 for j in tour])
    assert  num == 1, f'Vertex {i} repeats {num} times in tour'
for (i, j) in constraints:
    assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (3 points)')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/josiahroa/workspace/cs-masters/5414-greedy-algorithms/karatsuba/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/240bbf6760f94421a03e4b79afbb70d1-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/240bbf6760f94421a03e4b79afbb70d1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 29 COLUMNS
At line 178 RHS
At line 203 BOUNDS
At line 232 ENDATA
Problem MODEL has 24 rows, 24 columns and 80 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9.5 - 0.00 seconds
Cgl0003I 2 fixed, 0 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 10 strengthened rows, 0 substitutions
Cgl0003I 1 fixed, 0 tightened bounds, 8 strengthened rows,

In [68]:
cost_matrix=[ [None,3,4,3,5],
             [1, None, 2,4, 1],
             [2, 1, None, 5, 4],
             [1, 1, 5, None, 4],
             [2, 1, 3, 5, None] ]
n=5
constraints = [(4,3),(1,2)]
tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
i = 0
tour_cost = 0
for j in tour[1:]:
    tour_cost += cost_matrix[i][j]
    i = j
tour_cost += cost_matrix[i][0]
print(f'Tour:{tour}')
print(f'Cost of your tour: {tour_cost}')
assert abs(tour_cost-13) <= 0.001, 'Expected cost was 13'
for i in range(n):
    num = sum([1 if j == i else 0 for j in tour])
    assert  num == 1, f'Vertex {i} repeats {num} times in tour'
for (i, j) in constraints:
    assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (3 points)')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/josiahroa/workspace/cs-masters/5414-greedy-algorithms/karatsuba/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/8a81d0e2e2b846d5bb48de49442f7aba-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/8a81d0e2e2b846d5bb48de49442f7aba-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 29 COLUMNS
At line 178 RHS
At line 203 BOUNDS
At line 232 ENDATA
Problem MODEL has 24 rows, 24 columns and 80 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9.5 - 0.00 seconds
Cgl0003I 2 fixed, 0 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 10 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 10 strengthened rows

In [69]:
from random import uniform, randint

def create_cost(n):
    return [ [uniform(0, 5) if i != j else None for j in range(n)] for i in range(n)]

for trial in range(20):
    print(f'Trial # {trial}')
    n = randint(6, 11)
    cost_matrix = create_cost(n)
    constraints = [(1, 3), (4, 2), (n-1, 1), (n-2, 2)]
    tour = tsp_with_extra_constraints(n, cost_matrix, constraints)
    i = 0
    tour_cost = 0
    for j in tour[1:]:
        tour_cost += cost_matrix[i][j]
        i = j
    tour_cost += cost_matrix[i][0]
    print(f'Tour:{tour}')
    print(f'Cost of your tour: {tour_cost}')
    for i in range(n):
        num = sum([1 if j == i else 0 for j in tour])
        assert  num == 1, f'Vertex {i} repeats {num} times in tour'
    for (i, j) in constraints:
        assert tour.index(i) < tour.index(j), f'Tour does not respect constraint {(i,j)}'
print('Test Passed (10 points)')

Trial # 0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/josiahroa/workspace/cs-masters/5414-greedy-algorithms/karatsuba/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/93c6f8dff9a54f55baa2d93e349787aa-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/93c6f8dff9a54f55baa2d93e349787aa-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 41 COLUMNS
At line 270 RHS
At line 307 BOUNDS
At line 348 ENDATA
Problem MODEL has 36 rows, 35 columns and 128 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 6.05494 - 0.00 seconds
Cgl0003I 3 fixed, 0 tightened bounds, 19 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 st